# Análisis Exploratorio: San Francisco Bay Bike Rental

https://www.kaggle.com/benhamner/sf-bay-area-bike-share


### Aclaración: Todos las imagenes de mapas generadas tienen su respectivo código en nuestro repositorio de github. Al dejar el código generado en este notebook no se podian ver las visualizaciones ni los mapas

repositorio: https://github.com/hernantain/Analisis-San-Francisco

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pandas.tseries.holiday import USFederalHolidayCalendar
%matplotlib inline

In [2]:
trip_meta = pd.read_csv('data/trip.csv')
weather_meta = pd.read_csv('data/weather.csv')

In [3]:
pd.options.mode.chained_assignment = None

### Convierto las fechas a datetime

In [4]:
trip_meta['start_date'] = pd.to_datetime(trip_meta['start_date'])

KeyboardInterrupt: 

In [ ]:
trip_meta['end_date'] = pd.to_datetime(trip_meta['end_date'])

In [ ]:
import datetime
from datetime import date,time

In [ ]:
#Saco los horarios de las fechas para analizar primero los viajes que son realizados durante el mismo dia
trip_meta['start_date2'] = trip_meta.start_date.apply(lambda x: datetime.datetime(x.year, x.month, x.day,0,0))
trip_meta['end_date2'] = trip_meta.end_date.apply(lambda x: datetime.datetime(x.year, x.month, x.day,0,0))

In [ ]:
#Paso las duraciones de los viajes a minutos
trip_meta.duration = trip_meta.duration.apply(lambda x: x/60.0)

In [ ]:
trip_meta.rename(columns={'duration':'duration(min)'},inplace=True)

In [ ]:
trip_meta.info()

In [ ]:
same_day = trip_meta.loc[trip_meta.start_date2 == trip_meta.end_date2,['id','duration(min)','start_date2','end_date2','start_station_name','start_station_id','end_station_name','end_station_id']]

In [ ]:
print len(trip_meta) - len(same_day)

#### Entendemos que como son 2099 los viajes que empiezan un dia y terminan en otro (insignificativo frente a la enorme cantidad de registros) ante la posibilidad de que haya viajes de mas de un dia, o muy largos preferimos filtrarlos.

In [ ]:
same_day.info()

In [ ]:
same_day.describe()

### Filtro los viajes que duran menos de 5 minutos

In [ ]:
trips_longer = same_day.loc[same_day['duration(min)'] > 5,:]

In [ ]:
trips_longer.info()

### Son 125455 los viajes que duran menos de 5 min

In [ ]:
print len(same_day) - len(trips_longer)

In [ ]:
# Convierto a datetime la fecha en el DF weather
weather_meta['date'] = pd.to_datetime(weather_meta['date'])

# Viajes segun las condiciones climaticas

## ¿Influye la temperatura en la cantidad de viajes realizados?

In [ ]:
temperatures = weather_meta[['date','max_temperature_f','mean_temperature_f','min_temperature_f']]

In [ ]:
# Paso la temperatura a Celsius
temperatures.max_temperature_f = temperatures.max_temperature_f.apply(lambda x: (x-32) / 1.8)
temperatures.mean_temperature_f = temperatures.mean_temperature_f.apply(lambda x: (x-32) / 1.8)
temperatures.min_temperature_f = temperatures.min_temperature_f.apply(lambda x: (x-32) / 1.8)

In [ ]:
temperatures.rename(
    columns={'max_temperature_f':'max_temperature_c','mean_temperature_f':'mean_temperature_c','min_temperature_f':'min_temperature_c'},inplace=True)

In [ ]:
temperatures_avg = temperatures.groupby('date').mean()
temperatures_avg = pd.DataFrame(temperatures_avg.to_records())

In [ ]:
final = trips_longer[['id', 'duration(min)', 'start_date2', 'start_station_name','end_station_name']]
final.rename(columns={'start_date2':'date'},inplace=True)

In [ ]:
date_duration = final.loc[:,['duration(min)','date']].groupby('date').agg([np.size])
date_duration = pd.DataFrame(date_duration.to_records())
date_duration.columns=['date','# trips']

In [ ]:
duration_temperature = pd.merge(date_duration,temperatures_avg,how='inner',on='date')
duration_temperature['weekday'] = duration_temperature.date.apply(lambda x: x.weekday())
duration_temperature_weekdays = duration_temperature\
                        .loc[(duration_temperature.weekday != 5) & (duration_temperature.weekday != 6),:]

duration_temperature_weekdays = duration_temperature_weekdays\
                        [['# trips','date','max_temperature_c','mean_temperature_c','min_temperature_c']]
    
trips_temperature = duration_temperature_weekdays.\
                groupby(pd.cut(duration_temperature_weekdays['mean_temperature_c'], np.arange(3,34))).sum()

f = plt.figure()
ax = trips_temperature['# trips'].plot.bar(figsize=(15,8),colormap='autumn',layout=('# trips','Mean Temperature (C)'), ax=f.gca())
ax.set_ylabel('Viajes',size=20.0,labelpad=20.0);
ax.set_xlabel('Temperatura Promedio (C)',size=20.0,labelpad=20.0);
ax.set_facecolor('white')
ax.grid(color='k',axis='y' ,linestyle='dotted', linewidth=0.1)
ax.set_title('Cantidad de alquileres segun la temperatura los dias de semana',size=20.0)
plt.show()

Para realizar este análisis hicimos un promedio de las temperaturas medias registradas por cada día (son 5, correspondientes a los 5 barrios distintos). 
De esta forma podemos ver en el grafico generado que la mayor cantidad de viajes se da cuando la temperatura media por día es una temperatura agradable y no es ni muy elevada ni muy baja (entre 14° y 21° Celsius). 

# --------------------------------------------------------------------------------------------------

### ¿Influye la lluvia en la cantidad de viajes realizados durante la semana?

In [ ]:
precipitation = weather_meta[['date','precipitation_inches']]

In [ ]:
precipitation.columns = ['date','inches']

In [ ]:
precipitation = precipitation.loc[precipitation.inches != 'T',:]
precipitation.inches = pd.to_numeric(precipitation.inches)

In [ ]:
precipitation = precipitation.groupby('date').mean()
precipitation = pd.DataFrame(precipitation.to_records())

In [ ]:
precipitation_trips = pd.merge(date_duration,precipitation,how='inner',on='date')

In [ ]:
precipitation_trips = precipitation_trips[['# trips','date','inches']]

In [ ]:
precipitation_trips['weekday'] = precipitation_trips.date.apply(lambda x: x.weekday())
precipitation_trips = precipitation_trips.loc[precipitation_trips.weekday < 5,:]

In [ ]:
precipitation_trips_plot = precipitation_trips.\
                groupby(pd.cut(precipitation_trips['inches'], np.arange(0, 1.1, 0.045))).sum()

f = plt.figure()
ax = precipitation_trips_plot['# trips'].plot.bar(figsize=(15,8),colormap='seismic',ax=f.gca())
ax.set_ylabel('Viajes',size=20.0,labelpad=20.0);
ax.set_xlabel('Precipitacion (pulgadas)',size=20.0,labelpad=20.0);
ax.set_facecolor('white')
ax.grid(color='k',axis='y' ,linestyle='dotted', linewidth=0.1)
ax.set_title('Cantidad de alquileres segun la lluvia los dias de semana',size=20.0)
plt.show()

El grafico de arriba muestra de una forma muy clara que la mayor cantidad de viajes se realizan cuando no hay precipitaciones o las mismas son muy bajas.

De todos modos, se encuentran algunos registros cuando la precipitación es mas elevada. Esto puede deberse a que los viajes fueron realizados un día de lluvia pero en un momento del día en que aún no llovía.

# ------------------------------------------------------------------------------------------------------

## Viajes según la nubosidad


In [ ]:
cloudy = weather_meta[['date','cloud_cover']].groupby('date').mean()
cloudy_avg = pd.DataFrame(cloudy.to_records())

In [ ]:
cloudy_stats = pd.merge(final,cloudy_avg,how='inner',on='date')

## ¿Cual es el porcentaje de viajes en días de nubosidad elevada?

In [ ]:
total_cloudy = cloudy_stats.count()
travels_cloudy = cloudy_stats.loc[cloudy_stats.cloud_cover > 4].count() / cloudy_stats.count()
print travels_cloudy['id'] * 100

Los días de nubosidad elevada, se utiliza en menor cantidad el servicio de rental.

##### Aclaración: se consideró un día de nubosidad elevada cuando en esta escala que va de 0 a 8, es mayor a 4 el valor.

## ¿Influye la nubosidad en el largo de los viajes?

In [ ]:
travels_cloudy = cloudy_stats.loc[cloudy_stats.cloud_cover > 4]
fig = plt.figure()
ax = plt.subplot(111)
travels_cloudy.plot.scatter('duration(min)','cloud_cover',alpha=0.32,figsize=(12,8),ax=ax);
ax.set_ylabel('Nubosidad',size=10.0,labelpad=20.0)
ax.set_xlim(0,400);
ax.set_title('Duracion de viajes segun la nubosidad', fontsize=15)

En los días de nubosidad elevada, se ve como a partir que aumenta la nubosidad, disminuye la cantidad de viajes de mayor duración, siendo este parámetro un factor importante a la hora de decidir si utilizar el servicio o no, por las probables posibilidades de que precipite.

Cuanto más nublado se presente el día más cortos serán los viajes en ese día.

In [ ]:
# Cloud cover vs duration
ax = cloudy_stats.groupby('cloud_cover').mean()['duration(min)'].plot(figsize=(14,4));
ax.set_facecolor('white')
ax.set_ylabel('Duracion Promedio(min)',size=10.0,labelpad=20.0)
ax.set_xlabel('Nubosidad',size=10.0,labelpad=20.0)
ax.set_ylim(0,35)
ax.set_title('Promedio de duracion de viajes segun la nubosidad', fontsize=15)

# ¿Gana popularidad el servicio a lo largo de los años?

In [ ]:
mitad = datetime.datetime(2014,8,29)
trips_year = trips_longer['duration(min)'].groupby(trip_meta.start_date < mitad).agg([np.size])
f = plt.figure()
ax = trips_year.plot.barh(figsize=(12,8))
ax.set_xlabel('Viajes',size=15.0,labelpad=20.0)
ax.set_ylabel('Anio',size=15.0,labelpad=20.0)
ax.tick_params('both', colors='k',labelsize=15.0)
ax.grid(color='k',axis='x' ,linestyle='dotted', linewidth=0.1)
ax.set_yticklabels(['8/2014-8/2015','8/2013-8/2014'])
ax.set_facecolor('white')
ax.set_title('Cantidad de viajes segun el anio',size=20.0)
plt.show()

Luego de visualizar el gráfico que se encuentra arriba podemos llegar a la conclusión que de los 2 años de registros que tenemos, el segundo año se realizaron mas viajes que en el primero. Por lo que podemos considerar que el servicio de alquiler de bicicletas esta ganando popularidad entre los habitantes de San Francisco.

# ¿Varía el alquiler de bicicletas segun el mes del año?

In [ ]:
trips_longer['month'] = trips_longer.start_date2.apply(lambda x: x.month)

In [ ]:
import matplotlib.patches as mpatches

In [ ]:
f = plt.figure()
ax = trips_longer.groupby('month').size().plot(kind='bar',figsize=(15,8))
ax.set_ylabel('Viajes',size=15.0,labelpad=20.0)
ax.set_xlabel('Mes',size=15.0,labelpad=20.0)
ax.set_facecolor('white')
ax.set_title('Cantidad de viajes realizados segun el mes del anio', color='black',size=15.0)
red_patch = mpatches.Patch(color='r', label='Verano');yellow_patch = mpatches.Patch(color='y', label='Otonio')
cyan_patch = mpatches.Patch(color='c', label='Invierno');green_patch = mpatches.Patch(color='g', label='Primavera');
plt.legend(handles=[red_patch,yellow_patch,cyan_patch,green_patch])
ax.grid(color='k', linestyle='dotted',axis='y',linewidth=0.1)
ax.get_children()[0].set_color('c');ax.get_children()[1].set_color('c');ax.get_children()[2].set_color('g')
ax.get_children()[3].set_color('g');ax.get_children()[4].set_color('g');ax.get_children()[5].set_color('r')
ax.get_children()[6].set_color('r');ax.get_children()[7].set_color('r');ax.get_children()[8].set_color('y')
ax.get_children()[9].set_color('y');ax.get_children()[10].set_color('y');ax.get_children()[11].set_color('c')
ax.set_xticklabels(['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre'])
plt.show()

En el grafico se observa claramente como la mayor cantidad de viajes se da en los meses con una temperatura mas elevada mientras que decrece en los meses mas fríos. Esto coincide con el grafico de las temperaturas.


# ¿Cuáles son los horarios pico del servicio?

In [ ]:
trip_meta['hour'] = trip_meta.start_date.apply(lambda x: x.hour)

In [ ]:
f = plt.figure()
ax = trip_meta.groupby('hour').size().plot(kind='bar',figsize=(15,8))
ax.get_children()[8].set_color('r');ax.get_children()[17].set_color('r')
ax.grid(color='k', linestyle='dotted',axis='y' ,linewidth=0.1)
ax.set_ylabel('Viajes',size=15.0,labelpad=20.0)
ax.set_xlabel('Hora',size=15.0,labelpad=20.0)
ax.set_title('Cantidad de viajes realizados segun la hora del dia', color='black',size=15.0)
ax.set_facecolor('white')
red_patch = mpatches.Patch(color='r', label='Horas pico')
plt.legend(handles=[red_patch])
plt.show()

Analizando los viajes realizados a lo largo de toda la semana se observa que los horarios picos del servicio son a la hora de ir y volver del trabajo (8 y 17 hs). 

Mientras que obviamente el servicio no tiene casi uso a la madrugada y es significantemente menor durante las horas laborales.

# ¿Cuál es la franja horaria mas popular los fines de semana?

In [ ]:
weekend_hour = trip_meta[['start_date']]
weekend_hour['hour'] = weekend_hour.start_date.apply(lambda x: x.hour)
weekend_hour['weekday'] = weekend_hour.start_date.apply(lambda x: x.weekday())
weekend_hour = weekend_hour.loc[weekend_hour.weekday>4,:]

In [ ]:
def ax_paint_area(min,max):
    for x in range(min,max):
        ax.get_children()[x].set_color('r');

In [ ]:
MIN_RUSH=11
MAX_RUSH=17

f = plt.figure()
plt.title('Viajes por hora los fines de semana', color='black', fontsize = 15)
ax = weekend_hour.groupby('hour').size().plot(kind='bar',figsize=(15,8))
ax.grid(color='k', linestyle='dotted',axis='y' ,linewidth=0.1)
ax_paint_area(MIN_RUSH,MAX_RUSH)
ax.set_ylabel('Viajes',size=15.0,labelpad=20.0)
ax.set_xlabel('Hora',size=15.0,labelpad=20.0)
ax.set_facecolor('white')
red_patch = mpatches.Patch(color='r', label='Horas pico')
plt.legend(handles=[red_patch])
plt.show()

En el gráfico vemos como la franja horaria más popular los fines de semana se encuentra entre las 11 y las 16 por lo que a diferencia del gráfico anterior, en donde había dos horarios muy marcados, está relacionado con que el fin de semana la gente no tiene que cumplir ningún horario y elige aprovechar el día en los horarios de más luz y mejor clima.

Los fines de semana, la gente utiliza mucho más el servicio en horarios durante la tarde que los días de semana.

# Cantidad de alquileres por dia y la duracion promedio de los mismos.

In [ ]:
dif_day_trips = final[['duration(min)','date']]

In [ ]:
dif_day_trips['weekday'] = dif_day_trips['date'].apply(lambda x: x.weekday())

In [ ]:
dif_day_trips = dif_day_trips[['duration(min)','weekday']]

In [ ]:
trips_by_day = dif_day_trips.groupby('weekday').agg([np.mean,np.size])
# Falta ordenar por dia.

In [ ]:
dif_day_trips_avg = dif_day_trips.groupby('weekday').agg([np.mean])
dif_day_trips_cant = dif_day_trips.groupby('weekday').agg([np.size])

## ¿Cuantos alquileres hay por día y que duración promedio tienen los mismos?

In [ ]:
f = plt.figure()
ax = dif_day_trips_cant.plot(kind='line',figsize=(16,8))
ax.plot(dif_day_trips_cant,'g-',linewidth=5.0)
ax.set_title('Cantidad de viajes y duracion promedio de los mismos segun el dia', color='black',size=20.0)
ax.set_facecolor('white')
ax.set_ylabel('Cantidad de Viajes',color='g',size=20.0,labelpad=20.0);
ax.set_xlabel('Dias',size=20.0,labelpad=20.0);
ax.tick_params('y', colors='g',labelsize=15.0)
ax.tick_params('x', colors='k',labelsize=15.0)
ax.set_ylim(0,100000)
ax.set_xmargin(1.0)
ax.grid(color='k', linestyle='dotted',linewidth=0.1)
ax2 = ax.twinx()
red_patch = mpatches.Patch(color='r', label='Avg Duration(min)');blue_patch = mpatches.Patch(color='b', label='# Trips')
ax2.plot(dif_day_trips_avg,'m-',linewidth=5.0)

ax2.set_xticklabels(['-','Lunes','Martes','Miercoles','Jueves','Viernes','Sabado','Domingo'])
ax2.set_ylabel('Duracion Promedio (Min)', color='m',size=15.0,labelpad=20.0)
ax2.tick_params('y', colors='m',labelsize=15.0)
ax2.set_ylim(0,40)
ax2.set_facecolor('white')
ax2.grid(b=False)
ax.legend().set_visible(False)
plt.show()

Se observa claramente como la duración promedio de los viajes es menor los dias de semana, asi mismo como la cantidad de viajes realizados durante la semana laboral es mayor. Probablemente se realicen muchos viajes para ir a trabajar de corta duracion. 

Durante el fin de semana la situacion es inversa. Hay una menor cantidad de viajes realizados pero la duración promedio de los mismos aumenta. Probablemente esto quiera decir que los viajes que se realizan el fin de semana se realizan por ocio y sin restricciones de tiempo(por ejemplo: pasear por la ciudad de San Francisco).

# ¿Como se comportan los viajes segun el tipo de subscripción?

In [ ]:
subscription_trips = trip_meta.loc[:,['duration(min)','subscription_type']]
subscription_trips = subscription_trips.groupby('subscription_type').agg([np.mean,np.size])
subscription_trips

## ¿Cual es la duración promedio de los alquileres según la subscripción?

In [ ]:
subscription_trips2 = trip_meta.loc[:,['duration(min)','subscription_type','start_date2']]
subscription_trips2['weekday'] = subscription_trips2['start_date2'].apply(lambda x: x.weekday())
subscription_trips2 = subscription_trips2.loc[:,['duration(min)','subscription_type','weekday']].\
                                                    groupby(['weekday','subscription_type']).agg([np.mean])
subscription_trips2 = pd.DataFrame(subscription_trips2.to_records())
subscription_trips2.columns = ['weekday','subscription_type','Avg Trip']
subscription_trips2 = pd.pivot_table(subscription_trips2,values='Avg Trip',columns='subscription_type',index='weekday')

f = plt.figure()
ax = subscription_trips2.plot.bar(figsize=(12,8))
plt.title('Duracion Promedio de los viajes segun el dia y el tipo de subscripcion', color='black')
ax.set_ylabel('Duracion Promedio (Min)');
ax.set_xticklabels(['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado','Domingo'])
ax.set_xlabel('Dias',size=20.0,labelpad=20.0);
ax.set_facecolor('white')
plt.show()

En este grafico se observa como los viajes mas largos a lo largo de toda la semana lo realizan aquellos usuarios que no estan subscriptos al servicio. 

El fin de semana se nota un incremento en la duración promedio de ambos tipos de usuarios, lo cual  tiene lógica y coincide con el analisis realizado anteriormente acerca de los dias.

## ¿Como se distribuyen los alquileres de las bicicletas según el tipo de usuario?

In [ ]:
subscription_trips3 = trip_meta.loc[:,['duration(min)','subscription_type','start_date2']]
subscription_trips3['weekday'] = subscription_trips3['start_date2'].apply(lambda x: x.weekday())
subscription_trips3 = subscription_trips3.loc[:,['duration(min)','subscription_type','weekday']].\
                                        groupby(['weekday','subscription_type']).agg([np.size])
subscription_trips3 = pd.DataFrame(subscription_trips3.to_records())
subscription_trips3.columns = ['weekday','subscription_type','trips']
subscription_trips3 = pd.pivot_table(
                            subscription_trips3,values='trips',columns='subscription_type',index='weekday')
subscription_trips3 = subscription_trips3.divide(subscription_trips3.sum(axis=1),axis=0)

f = plt.figure()
ax = subscription_trips3.plot(kind='area', stacked=True,figsize=(16,8),ax=f.gca())
handles, labels = ax.get_legend_handles_labels()
ax.set_xticklabels(['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado','Domingo'])
ax.set_yticklabels(['0%','20%','40%','60%','80%','100%'])
ax.set_ylabel('Porcentaje',size=20.0,labelpad=20.0);
ax.set_xlabel('Dias',size=20.0,labelpad=20.0);
ax.set_title('Porcentaje de viajes segun la subscripcion a lo largo de la semana', color='black',size=20.0)
ax.set_ylim(0,1)
plt.legend(handles[::-1], labels[::-1],loc='center left', bbox_to_anchor=(1.0, 0.5))
plt.show()

En este grafico de área podemos ver como durante la semana laboral, el 80% de los viajes los realizan aquellos usuarios subscriptos al servicio mientras que en el fin de semana sube la cantidad de viajes realizados por los usuarios no subscriptos. 

Lo cual sigue coincidiendo con nuestra teoria de que durante la semana la mayor cantidad de viajes que se realizan son relacionados al trabajo mientras que en el fin de semana se realizan mas viajes por ocio.

## ¿Cuáles son las estaciones más populares?

In [ ]:
station_stats = pd.concat([trip_meta['start_station_name'], trip_meta['end_station_name']])

In [ ]:
# TOP 20 Estaciones mas populares
f = plt.figure()
ax = station_stats.value_counts().head(20).plot(kind='bar',rot=90,figsize=(12,8));
plt.title('TOP 20 de estaciones mas visitadas', color='black')
ax.set_ylabel('Viajes');
ax.grid(color='k',axis='y' ,linestyle='dotted',linewidth=0.1)
ax.set_xlabel('Estaciones');
ax.set_facecolor('white')
plt.show()

# HeatMap de las estaciones mas usadas en San Francisco
![](images/heatmap-codigo.png)

![](images/heatmap-SanFrancisco-Nombres.png)

![](images/heatmap-SanFranciscoCaltrain.png)

## ¿Cuáles son las estaciones más usadas según el día?

In [ ]:
popular_stations = trip_meta[['start_date','start_station_name','duration(min)']]

In [ ]:
# Filtro los viajes que duran menos de 5 minutos
popular_stations = popular_stations.loc[popular_stations['duration(min)'] > 5,:]

In [ ]:
popular_stations.rename(columns={'start_date':'date','start_station_name':'station'},inplace=True)

In [ ]:
popular_stations['weekday'] = popular_stations.date.apply(lambda x: x.weekday())

In [ ]:
popular_stations = popular_stations[['station','duration(min)','weekday']]

In [ ]:
popular_stations = popular_stations.groupby(['station','weekday']).agg([np.size])
popular_stations = pd.DataFrame(popular_stations.to_records())
popular_stations.columns = ['station','weekday','trips']
popular_stations = pd.pivot_table(popular_stations,values='trips',columns='weekday',index='station')
fig, ax = plt.subplots(figsize=(10,20))
ax = sns.heatmap(popular_stations)
ax.set_xticklabels(['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado','Domingo'])
ax.set_title("Popularidad de las estaciones segun el dia", fontsize=15)
ax

En este heatmap se ve claramente como hay estaciones que de lunes a viernes son extremadamente populares mientras que los fines de semana decrece su uso enormemente debido a su ubicación centrica y relacionada con áreas laborales.

Además, notamos la consistencia en nuestro análisis al ver que las estaciones más marcadas en el mapa de calor condicen con el TOP 20 de estaciones populares realizado anteriormente.

## ¿Cuáles son las estaciones mas populares los fines de semana?

In [ ]:
final.rename(columns={'name':'start_station_name'},inplace=True)
weekend_stations = final[['start_station_name','end_station_name','date']]
weekend_stations['weekday'] = weekend_stations.date.apply(lambda x: x.weekday())
weekend_stations = weekend_stations.loc[weekend_stations.weekday>4,:]
weekend_stations = pd.concat([weekend_stations['start_station_name'], weekend_stations['end_station_name']])
f = plt.figure()
ax = weekend_stations.value_counts().head(10).plot(kind='bar',rot=90,figsize=(12,8));
plt.title('Estaciones mas populares los fines de semana', color='black', fontsize=15)
ax.set_ylabel('Viajes');
ax.set_xlabel('Estaciones');
ax.set_facecolor('white')
ax.get_children()[0].set_color('r');ax.get_children()[1].set_color('r')
plt.show()



## ¿Cuál de las estaciones nuevas fue la mas usada?

In [ ]:
station_meta = pd.read_csv('data/station.csv')
station_meta['installation_date'] = pd.to_datetime(station_meta['installation_date'])

In [ ]:
last_year = datetime.datetime(2014,1,1)
new_stations = station_meta.loc[station_meta.installation_date >= last_year]

In [ ]:
final.rename(columns={'start_station_name':'name'},inplace=True)
new_stations = pd.merge(final,new_stations,how='inner',on='name')

In [ ]:
ax = new_stations[['name','installation_date']]\
                .groupby('name').count()\
                .rename(columns={'installation_date':'Viajes'})\
                .plot(figsize=(15,5));
ax.grid(color='k', linestyle='dotted',axis='y' ,linewidth=0.3)
ax.set_title("Cantidad de viajes en estaciones nuevas", fontsize=15)
ax.set_facecolor("white")
ax

Broadway St at Battery St fue de las 3 estaciones más nuevas la más frecuentada.

#### Aclaración: se consideró como nuevas a las estaciones cuya fecha de instalación databan a partir del 2014 (último año del set de datos)

## ¿Cuál es el promedio de docks por estación?

In [ ]:
# Quiero ver cual es el promedio de docks en cada estacion
station_meta['dock_count'].mean()

## ¿Qué cantidad de docks posee la más usada de las nuevas?

In [ ]:
# Ahora quiero ver que pasa en Broadway St at Battery (de las nuevas, la mas usada)
station_meta.loc[station_meta.name == 'Broadway St at Battery St']['dock_count']

Como la estación más usada de las nuevas tiene un número de docks levemente por debajo del promedio, podría ser una sugerencia útil, agregarle algunos docks más a la estación ya que desde que fue instlada ha sido frecuentada. Tambien podría ser de utilidad para el servicio chequear la zona de esta estación para agregar alguna nueva cercana dada su popularidad.

## ¿Cuál es la ciudad que mas usa el servicio?

In [ ]:
popular_city = pd.merge(final,station_meta,how='inner',on='name')
ax = popular_city.groupby('city').count()['id_x'].plot(kind='barh',figsize=(14,4));
ax.set_title("Cantidad de viajes por ciudad", fontsize=15)
ax.set_xlabel('Viajes',size=10.0,labelpad=20.0)
ax.set_facecolor("white")

Es lógico que así lo sea, el servicio es muy utilizado en la ciudad de San Francisco como se ve también en el heatmap de abajo, ya que es la ciudad con más movimiento, más habitantes de las 5 involucradas y también la más visitada.

# HeatMap de las estaciones mas usadas en todas las ciudades (equivale con la  cantidad de viajes)
![](images/heatmap-ciudades.png)

## Por amplia diferencia San Francisco es la ciudad que más uso le da al servicio.

# -------------------------------------------------------------------------------------------------------- 

## Analisis segun fechas

In [ ]:
dates_df = pd.DataFrame()
dates_df['date'] = trip_meta['start_date'].dt.date
dates_df['trips'] = dates_df.groupby('date')['date'].transform('count')
dates_df = dates_df.drop_duplicates('date')
dates_df.reset_index(drop = True, inplace = True)

In [ ]:
dates_df['date'] = pd.to_datetime(dates_df['date'])
dates_df.dtypes

In [ ]:
import matplotlib.dates as mdates
f = plt.figure()
plt.title('Viajes por dia a lo largo del periodo', color='black')
ax = ((dates_df.set_index('date')).loc[:,'trips']).plot(kind='line', stacked=True,figsize=(16,8),ax=f.gca())
handles, labels = ax.get_legend_handles_labels()
ax.set_ylabel('Viajes');
ax.set_xlabel('Dia');
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
plt.show()

Al ser muy extenso el intervalo de fechas, el grafico no es tan detallado. Mas adelante se hace enfasis en una parte del mismo.

## ¿Qué tanto viaja la gente en feriados a comparación con un día corriente?

In [ ]:
#Busco los feriados en el tiempo transcurrido
calendar = USFederalHolidayCalendar()
holidays = calendar.holidays(start=dates_df.date.min(), end=dates_df.date.max())

In [ ]:
dates_df['holiday'] = dates_df.date.isin(holidays)

In [ ]:
dates_df['weekend'] = dates_df['date'].dt.dayofweek > 4

In [ ]:
holidays_mean = dates_df.groupby('holiday')['trips'].mean()
ax = holidays_mean.plot.bar(figsize=(12,8))
ax.set_xticklabels(['No feriados','Feriados'])
ax.set_title("Promedio de viajes en feriados y no feriados", fontsize=15)
ax.set_ylabel('Viajes Promedio',size=15.0,labelpad=20.0)
ax.set_xlabel('Tipo de Dia',size=15.0,labelpad=20.0)
ax.set_ylim(0,1200)
ax.set_facecolor("white")

Como se ve en el gráfico, en los feriados la gente viaja aproximadamente la mitad de las veces que en los días corrientes. Esto nos da un indicio de que el servicio es frecuente como medio de transporte para ir a trabajar o para transportarse por la ciudad en días normales.

# Análisis anual

### Período 2014

In [ ]:
f = plt.figure()
plt.title('Cantidad de viajes por dia',fontsize=20)
ax = (dates_df.loc[:,['trips','date']].set_index('date')).plot(figsize=(25,10),ax=f.gca())
ax.set_ylabel('Viajes');
ax.set_xlabel('Dia');
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
ax.axvspan(*mdates.datestr2num(['1/1/2014', '12/31/2014']), color='red', alpha=0.5)
plt.show()

Del grafico anterior tomamos el intervalo correspondiente al 2014.

In [ ]:
trips_2014 = dates_df.loc[(dates_df['date'] >= datetime.date(2014,1,1)) & (dates_df['date'] < datetime.date(2015,1,1))]
trips_2014.sort_values('date',inplace=True)

## ¿Qué sucede en los feriados y fines de semana en un año? 

In [ ]:
holidays_2014 = trips_2014.loc[trips_2014['holiday']]
weekends_2014 = trips_2014.loc[trips_2014['weekend']]
plt.figure(figsize=(20,10))
weekends_plt, = plt.plot(weekends_2014['date'],weekends_2014['trips'],'gD',label='Weekend')
holidays_plt, = plt.plot(holidays_2014['date'],holidays_2014['trips'],'rD',label='Holiday')
plt.plot(trips_2014['date'],trips_2014['trips'],'-')
plt.legend(handles=[weekends_plt,holidays_plt])
plt.title("Viajes a lo largo del ano 2014", fontsize=20)
plt.show()

Se observa que los picos bajos del grafico corresponden a los fines de semana, en los que la cantidad de viajes es considerablemente menor a los dias laborales. Para los feriados ocurre algo similar, excepto para dos fechas que se analizan a continuacion.

### ¿Los feriados con mas viajes son una constante para todos los años?

In [ ]:
holidays_2014.sort_values('trips',inplace=True,ascending=False)
holidays_2014.head(2)

In [ ]:
trips_2013 = dates_df.loc[(dates_df['date'] >= datetime.date(2013,1,1)) & (dates_df['date'] < datetime.date(2014,1,1))]
holidays_2013 = trips_2013.loc[trips_2013['holiday']]
holidays_2013.sort_values('trips',inplace=True,ascending=False)
holidays_2013.head(2)

Para el año 2015 no hay registros de estas fechas, pero se observa que la tendencia es que el segundo lunes de octubre('Columbus Day') y el 11 de noviembre('Veteran's Day') son los feriados con más viajes en el año.

## ¿Qué bicicletas tienen mas tiempo recorrido?

In [ ]:
bike_longer = trip_meta.loc[trip_meta.start_date2 == trip_meta.end_date2,['duration(min)','bike_id']]
bike_longer = bike_longer.loc[bike_longer['duration(min)'] > 5,:]

In [ ]:
# TOP 10 bicicletas con mas tiempo recorrido (para posible mantenimiento)
top10 = bike_longer.groupby('bike_id').sum().sort_values('duration(min)', ascending=False).head(10)
ax = top10.plot(kind='bar',figsize=(14,5));
ax.set_title("Tiempo recorrido de las bicis mas usadas", fontsize=15)
ax.set_facecolor("white")
ax.set_ylabel('Viajes',size=20.0,labelpad=20.0)
ax.set_xlabel('Bike ID',size=20.0,labelpad=20.0)
ax.set_ylim(25000,31000)
ax.grid(color='k', linestyle='dotted',axis='y' ,linewidth=0.5)
ax

#### Esta información podría ser relevante para el servicio a la hora de realizar una tarea de mantenimiento.

# ¿Cuales son los viajes mas frecuentes?

In [ ]:
trip_meta2 = trip_meta[['start_station_name','end_station_name','duration(min)']]
trip_meta2 = trip_meta2.groupby(['start_station_name','end_station_name']).agg([np.size])
trip_meta2 = pd.DataFrame(trip_meta2.to_records())
trip_meta2.columns = ['start_station','end_station','trips']
trip_meta2 = trip_meta2.sort_values('trips',ascending=False).head().reset_index()

In [ ]:
origen = pd.merge(trip_meta2,station_meta,how='left',left_on='start_station',right_on='name')

In [ ]:
origen = origen[['start_station','end_station','trips','lat','long']]

In [ ]:
origen = origen[['start_station','end_station','trips','lat','long']]

In [ ]:
origen_destino = pd.merge(origen,station_meta,how='left',left_on='end_station',right_on='name')

In [ ]:
origen_destino = origen_destino[['start_station','end_station','trips','lat_x','long_x','lat_y','long_y']]
origen_destino

## 1er viaje mas frecuente:
![](images/direction-layer1.png)

## 2do viaje mas frecuente:
![](images/direction-layer2.png)

## 3er viaje mas frecuente:
![](images/direction-layer3.png)

### Este ultima imagen corresponde al 4to y 5to viaje mas frecuente ida y vuelta(ver tabla):
![](images/direction-layer4.png)